<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 5 Assignment: Computer Vision Neural Network**

**Student Name: Your Name**

# Google CoLab Instructions

This assignment will be most straightforward if you use Google CoLab, because it requires both PyTorch and YOLOv5 to be installed. It will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,course,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'course':course, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment Instructions

For this assignment, you will use YOLO running on Google CoLab.  I suggest that you run this assignment on CoLab because the example code below is already setup to get you started with the correct versions of  YOLO on PyTorch.

For this assignment you are provided with 10 image files that contain 10 different webcam pictures taken at the [Venice Sidewalk Cafe](https://www.westland.net/beachcam/) a WebCam that has been in opration since 1996.  You can find the 10 images here:

* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk2.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk3.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk4.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk5.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk6.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk7.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk8.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk9.jpg
* https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk10.jpg

You can see a sample of the WebCam here:

![alt text](https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg)

YOLO does quite well-recognizing objects in this webcam, as the following image illustrates.

![alt text](https://data.heatonresearch.com/data/t81-558/sidewalk/predictions.jpg)

You are to write a script that counts the number of certain objects in each of the images.  Specifically, you are looking for:

* person
* car
* bus

**Do not include any counts but these three.**

Your submitted data frame should also contain a column that identifies which image generated each row.  This column should be named **image** and contain integer numbers between 1 and 10.  There should be 10 rows in total.  The complete data frame should look something like this (not necessarily exactly these numbers).

|image|bus|car|person|
|-|-|-|-|
|1|1|7|18|
|2|0|7|23|
|3|0|1|13|
|...|...|...|...|


The following code sets up YOLO and then dumps the classification information for the first image.  This notebook only serves to get you started.  Read in all ten images and generate a data frame that looks like the following. Use the **submit** function as you did in previous assignments.

### Installing YOLO

YOLO is now available directly through PIP. The following commands install YOLO with PIP.

In [ ]:
!pip install ultralytics

### Running YOLO

Use this command and arguments to run YOLO, these values for iou and conf are necessary to get the same results as my solution file. I just use one image as a test, you will need to use all 10 in separate calls.

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
# TEST_IMAGE_FILE = 'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg'

TEST_IMAGE_FILE = [
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk2.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk3.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk4.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk5.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk6.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk7.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk8.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk9.jpg',
  'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk10.jpg']

# results = model(TEST_IMAGE_FILE,conf=0.1,iou=0.8)  # predict on an image

You will also find the Pandas loader that we used in the course material helpful:

In [ ]:
import pandas as pd

def results_to_dataframe(results):
    lookup = results[0].names
    boxes = results[0].boxes

    data = {
        'class': [int(item) for item in boxes.cls],
        'name': [lookup[int(item)] for item in boxes.cls],
        'xmin': [int(box[0]) for box in boxes.xyxy],
        'ymin': [int(box[1]) for box in boxes.xyxy],
        'xmax': [int(box[2]) for box in boxes.xyxy],
        'ymax': [int(box[3]) for box in boxes.xyxy],
    }

    return pd.DataFrame(data)


In [ ]:
categories = ['person', 'car', 'bus']

results_data = []

for i, img_url in enumerate(TEST_IMAGE_FILE, 1):
    # Run inference with specified conf and IoU
    results = model(img_url, conf=0.1, iou=0.8)

    # Convert results to dataframe using the provided function
    df_results = results_to_dataframe(results)

    # Count occurrences of each category
    person_count = len(df_results[df_results['name'] == 'person'])
    car_count = len(df_results[df_results['name'] == 'car'])
    bus_count = len(df_results[df_results['name'] == 'bus'])

    # Append the results to the list
    results_data.append({
        'image': i,
        'person': person_count,
        'car': car_count,
        'bus': bus_count
    })

    # Build the df_submit for each image
    df_submit = results_to_dataframe(results)
    # print(f"Results for Image {i}:")
    # print(df_submit)
    # print("\n")

df_final = pd.DataFrame(results_data)



Found https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk1.jpg locally at sidewalk1.jpg
image 1/1 /content/sidewalk1.jpg: 352x640 18 persons, 7 cars, 1 bus, 1 traffic light, 8.4ms
Speed: 2.8ms preprocess, 8.4ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 640)

Found https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk2.jpg locally at sidewalk2.jpg
image 1/1 /content/sidewalk2.jpg: 352x640 23 persons, 1 bicycle, 7 cars, 1 truck, 1 bench, 1 umbrella, 10.2ms
Speed: 2.3ms preprocess, 10.2ms inference, 1.7ms postprocess per image at shape (1, 3, 352, 640)

Found https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk3.jpg locally at sidewalk3.jpg
image 1/1 /content/sidewalk3.jpg: 352x640 13 persons, 1 car, 4 umbrellas, 7.4ms
Speed: 1.9ms preprocess, 7.4ms inference, 1.8ms postprocess per image at shape (1, 3, 352, 640)

Found https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk4.jpg locally at sidewalk4.jpg
image 1/1 /content/sidewalk4.

I built the following function from the code presented in the course module. The function combines some of the code from the module to accept an image and return what YOLO recognizes. Make sure to use the same two thres_xxx values I provided below to match the results that I got.

### Solution

In [ ]:
# df_submit = results_to_dataframe(results)
# df_submit

In [ ]:
key = ""

file= '/content/drive/My Drive/Colab Notebooks/assignment_ZhijiangLi_class5.ipynb'

# submit_df = df_submit
submit_df = df_final

In [ ]:
# Submit
submit(source_file=file,data=[submit_df],key=key,course='t81-558',no=5)

Success: Submitted Assignment 5 (t81-558) for l.zhijiang:
You have submitted this assignment 3 times. (this is fine)
No errors, warnings, or notes on your data. Rock on! You will probably do well, but no guarantee. :-)
